# Titanic traning without using pipeline

In [18]:
import pandas as pd
import numpy as np

In [19]:
df= pd.read_csv("/content/tatinic-train.csv")

In [20]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [21]:
from sklearn.model_selection import train_test_split 
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import MinMaxScaler 
from sklearn.tree import DecisionTreeClassifier 

In [22]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [23]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [24]:
X_train,X_test,y_train,y_test=train_test_split (df.drop('Survived',axis=1),df['Survived'],test_size=0.2,random_state=35)

In [25]:
X_train.shape

(712, 7)

In [26]:
X_test.shape

(179, 7)

In [27]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
465,3,male,38.0,0,0,7.0500,S
202,3,male,34.0,0,0,6.4958,S
317,2,male,54.0,0,0,14.0000,S
599,1,male,49.0,1,0,56.9292,C
438,1,male,64.0,1,4,263.0000,S


In [28]:
X_train.isnull().sum()

Pclass        0
Sex           0
Age         140
SibSp         0
Parch         0
Fare          0
Embarked      1
dtype: int64

In [29]:
#Applying SimpleImputer() -> Age, Embarked

si_age=SimpleImputer()
si_embarked = SimpleImputer(strategy='most_frequent')

X_train_age=si_age.fit_transform(X_train[['Age']])
X_test_age=si_age.fit_transform(X_test[['Age']])

X_train_embarked=si_embarked.fit_transform(X_train[['Embarked']])
X_test_embarked=si_embarked.fit_transform(X_test[['Embarked']])

In [30]:
X_train_embarked.shape

(712, 1)

In [31]:
#Applying OneHotEncoder () -> Sex, Embarked

ohe_sex=OneHotEncoder(sparse=False,handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse=False,handle_unknown='ignore')

X_train_sex=ohe_sex.fit_transform(X_train[['Sex']])
X_test_sex=ohe_sex.fit_transform(X_test[['Sex']])



In [32]:
X_train_sex

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [33]:
X_train_embarked=ohe_embarked.fit_transform(X_train_embarked)
X_test_embarked=ohe_embarked.transform(X_test_embarked)

In [34]:
X_train_embarked

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [35]:
X_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
465,3,male,38.0,0,0,7.0500,S
202,3,male,34.0,0,0,6.4958,S


In [36]:
X_train_rem=X_train.drop(columns=['Sex','Age','Embarked'])
X_test_rem=X_test.drop(columns=['Sex','Age','Embarked'])

In [37]:
X_train_rem.head()

,Pclass,SibSp,Parch,Fare
465,3,0,0,7.0500
202,3,0,0,6.4958
317,2,0,0,14.0000
599,1,1,0,56.9292
438,1,1,4,263.0000


In [38]:
X_train_transformed = np.concatenate((X_train_rem,X_train_age,X_train_embarked,X_train_sex),axis=1)
X_test_transformed = np.concatenate((X_test_rem,X_test_age,X_test_embarked,X_test_sex),axis=1)

In [39]:
X_train_transformed.shape

(712, 10)

In [40]:
# Traning DecisionTreeClassifier 

clf = DecisionTreeClassifier ()
clf.fit(X_train_transformed,y_train)

DecisionTreeClassifier()

In [41]:
y_pred = clf.predict(X_test_transformed)

In [42]:
y_pred 

array([0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 1])

In [43]:
from sklearn.metrics import accuracy_score 

accuracy_score (y_test,y_pred)

0.7597765363128491

In [44]:
import pickle

In [45]:
# we also store ohe_sex and ohe_embarked bacause we are using this transformer
pickle.dump(ohe_sex,open('models/ohe_sex.pkl','wb'))
pickle.dump(ohe_embarked,open('models/ohe_embarked.pkl','wb'))
pickle.dump(clf,open('models/clf.pkl','wb'))

#Prediction without pipeline

In [46]:
import pickle
import numpy as np

In [47]:
ohe_sex = pickle.load(open('/content/models/ohe_sex.pkl','rb'))
ohe_embarked = pickle.load(open('/content/models/ohe_embarked.pkl','rb'))
clf = pickle.load(open('/content/models/clf.pkl','rb'))

In [48]:
# Assume user input

# Pclass/gsender/age/SibSp/Parch/Fare/Embarked 

test_input  = np.array([2,'male',31.0,0,0,10.5,'S'], dtype=object).reshape(1,7)

In [49]:
test_input

array([[2, 'male', 31.0, 0, 0, 10.5, 'S']], dtype=object)

In [50]:
test_input_sex = ohe_sex.transform(test_input[:,1].reshape(1,1))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  "X does not have valid feature names, but"


In [51]:
test_input_sex

array([[0., 1.]])

In [52]:
test_input_embarked = ohe_embarked.transform(test_input[:,-1].reshape(1,1))

In [53]:
test_input_embarked

array([[0., 0., 1.]])

In [54]:
test_input_age = test_input[:,2].reshape(1,1)

In [57]:
test_input_transformed = np.concatenate((test_input[:,[0,3,4,5]],test_input_age,test_input_embarked,test_input_sex),axis=1)

In [58]:
test_input_transformed

array([[2, 0, 0, 10.5, 31.0, 0.0, 0.0, 1.0, 0.0, 1.0]], dtype=object)

In [60]:
test_input_transformed.shape

(1, 10)

In [61]:
clf.predict(test_input_transformed)

array([1])